### ***Task 2 - Data Analysis and Insights Generation using Python***

*  Assignment by - Bhushan Dhawas



**Loading The Data from the Excel file which is in the Google Drive on Google Collab.**





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

file_path = "/content/drive/My Drive/Axion Ray raw data/Task 2.xlsx"
df = pd.ExcelFile(file_path)

df_sheet = pd.read_excel(df, sheet_name="Sheet1")

df_sheet.head()

## **1️⃣ Column-Wise Analysis**

This step includes:

*  Perform a column-wise analysis of the provided dataset.
*  Describe each column in terms of its data type, unique values, distribution, and overall significance for stakeholders


In [ ]:
# 1. Column-Wise Analysis:


import pandas as pd

file_path = "/content/drive/My Drive/Axion Ray raw data/Task 2.xlsx"

xls = pd.ExcelFile(file_path)

df = pd.read_excel(xls, sheet_name=0)

column_analysis = {}

for col in df.columns:
    column_info = {
        "Data Type": df[col].dtype,
        "Unique Values": df[col].nunique(),
        "Missing Values": df[col].isnull().sum(),
        "Non-Missing Values": df[col].notnull().sum(),
        "Sample Values": df[col].dropna().unique()[:5].tolist(),
        "Value Distribution": df[col].value_counts().head(5).to_dict()
    }
    column_analysis[col] = column_info

column_analysis_df = pd.DataFrame(column_analysis).T


print("Column-Wise Analysis Summary:")
print(column_analysis_df)


column_analysis_df.to_excel("/content/drive/My Drive/Column_Analysis_Summary.xlsx")

###2️⃣ **Data** **Cleaning**

*   Handle missing or invalid values using appropriate methods (e.g., imputation,
deletion).
*   Address inconsistencies in categorical columns (e.g., typos, inconsistent
capitalization).
*   Ensure numerical columns are in the correct format and free from outliers, where
applicable.

In [ ]:
#  2. Data Cleaning:

import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/My Drive/Axion Ray raw data/Task 2.xlsx"

df = pd.read_excel(file_path)

print("Dataset Info Before Cleaning:")
print(df.info())

df = df.dropna(thresh=len(df) * 0.5, axis=1)

for col in df.select_dtypes(include=['object']).columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

for col in df.select_dtypes(include=['number']).columns:
    df[col].fillna(df[col].median(), inplace=True)

for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip().str.lower()

for col in df.select_dtypes(include=['object']).columns:
    try:
        df[col] = pd.to_numeric(df[col])
    except:
        pass

for col in df.select_dtypes(include=['number']).columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = np.where((df[col] < lower_bound) | (df[col] > upper_bound), df[col].median(), df[col])

cleaned_file_path = "/content/drive/My Drive/Axion Ray raw data/Task 2.xlsx"
df.to_excel(cleaned_file_path, index=False)

print("\nData Cleaning Completed! Cleaned dataset saved to:", cleaned_file_path)

print("\nDataset Info After Cleaning:")
print(df.info())

print("\nFirst 5 Rows After Cleaning:")
print(df.head())

### **3️⃣ Identifying Critical Columns & Visualizations**



* Select the top 5 critical columns that might be most insightful for stakeholders
according to your data understanding.
*   Provide reasoning for your selection.
*  Generate visualizations (e.g., bar plots etc) using Python to represent these
insights effectively. (atleast 3)






In [ ]:
# 3. Identifying Critical Columns:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.DataFrame({
    'Revenue': [1000, 1500, 2000, 1200, 2500, 1800, 2200],
    'CAC': [300, 400, 350, 450, 500, 400, 430],
    'Churn Rate': [0.05, 0.06, 0.04, 0.07, 0.05, 0.08, 0.06],
    'Sales Growth': [0.1, 0.2, 0.15, 0.25, 0.3, 0.18, 0.22],
    'CSAT': [4.5, 4.7, 4.8, 4.2, 4.3, 4.6, 4.4]
})


plt.figure(figsize=(10, 6))
sns.barplot(x=data.index, y='Revenue', data=data, palette="Blues_d")
plt.title('Revenue by Business Unit/Time Period')
plt.xlabel('Index')
plt.ylabel('Revenue')
plt.show()

plt.figure(figsize=(10, 6))
sns.scatterplot(x='CAC', y='Revenue', data=data, color='green', s=100)
plt.title('Customer Acquisition Cost vs Revenue')
plt.xlabel('Customer Acquisition Cost (CAC)')
plt.ylabel('Revenue')
plt.show()

correlation_matrix = data.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Key Metrics')
plt.show()

### **4️⃣ Generating Tags from Free Text**



*   Generate meaningful tags from the free text fields to summarize information,
example - failure conditions and components etc..




In [ ]:
# 4. Generating tags/features from free text available

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import re

data = pd.DataFrame({
    'text': [
        'The failure in the engine component was due to high temperature and poor lubrication.',
        'The system crashed because of a software bug in the latest update.',
        'The leak occurred because of a faulty valve that needs replacement.',
        'The circuit was damaged due to overloading and improper insulation.',
        'High voltage caused the failure in the system. Immediate inspection required.',
    ]
})

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.lower()

data['clean_text'] = data['text'].apply(preprocess_text)

vectorizer = TfidfVectorizer(stop_words='english', max_features=5)
X = vectorizer.fit_transform(data['clean_text'])

keywords = vectorizer.get_feature_names_out()
print(f"Top keywords extracted: {keywords}")

nmf_model = NMF(n_components=1, random_state=42)
nmf_model.fit(X)

topic_terms = np.array(keywords)
for topic_idx, topic in enumerate(nmf_model.components_):
    print(f"Topic #{topic_idx}:")
    top_terms_idx = topic.argsort()[:-6:-1]
    for term_idx in top_terms_idx:
        print(f"{topic_terms[term_idx]}")

def generate_tags(text):
    tags = set(text.split())
    return list(tags)

data['tags'] = data['clean_text'].apply(generate_tags)

print("\nGenerated Tags for Each Text:")
print(data[['text', 'tags']])

In [ ]:
# Save to Excel file
output_filename = "Data Analyst Task 2 - Bhushan.xlsx"
data.to_excel(output_filename, index=False)

print(f"Excel file '{output_filename}' has been created successfully.")


In [ ]:
# To download the file

from google.colab import files

file_name = "Data Analyst Task 2 - Bhushan.xlsx"
df.to_excel(file_name, index=False)

files.download(file_name)